In [11]:
# import libraries
import requests
from bs4 import BeautifulSoup, NavigableString
import json
import re

In [12]:
# scrapt the content on the given url
def scrap_content(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    content_block = soup.find('div', class_='entry-content').find_all()

    # topic for questions or topics
    # description for collecting data for topics
    # bundle for combining topic : desciption
    topic = []
    description = []
    bundle = {}

    for block in content_block:
      for line in block.contents:

        # if there is strong tag in line then that line contains topic
        if "<strong>" in str(line):
          if(len(topic)>0):
            bundle[topic[-1]] = description;
          description = []

          # remove values other than a-z, A-Z, 0-9, space
          t1 = re.sub(r'[^a-zA-Z0-9\s]+', '', line.text.strip())
          topic.append(t1)

        # if img tag contains in the line then we can skip that line
        elif "<img" not in str(line):
          line = str(line).replace('\n',' ').strip()

          # remove <br/> from the line
          br_tag_pattern = re.compile(r"<br\s*/?>", re.IGNORECASE)
          line = re.sub(br_tag_pattern, "", line)

          # remove values other than a-z, A-Z, 0-9, space and .
          line = re.sub(r'[^A-Za-z0-9.\s]+', '', line)
          if len(line)>0:
            description.append(line)

    # zip key and value into bundle
    bundle = {key.strip():value for (key, value) in bundle.items() if key!='' and len(value)>0}
    return bundle

In [13]:
# parse the webpage and return the extracted links
def get_page(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    notes_links = []

    # find link tags
    links = soup.find('div', class_='featured-snippet').ul.find_all('li')
    for i in links:
      notes_links.append(i.a['href'])
    return notes_links

In [14]:
url = "https://www.learncbse.in/cbse-notes-class-11-physics/"
links = get_page(url)
que_ans = {}

for i in range(len(links)):
  que_ans.update(scrap_content(links[i]))
for i,j in que_ans.items():
  que_ans[i] = ''.join(j)

In [16]:
# assign unique id for all the question answers and dump it into json file
data = {}
data["data"] = []
id = 1

for i,j in que_ans.items():
  temp = {}
  temp['id'] = id
  temp['question'] = i
  temp['answer'] = j
  data['data'].append(temp)
  id += 1

with open("/content/drive/MyDrive/PhysicsBuddy/data.json", "w") as file:
  json.dump(data, file)